In [4]:
from __future__ import print_function
import torch

import torch.nn as nn
import torch.nn.functional as F


from mnist import MNIST
import numpy as np

In [6]:
def load_dataset():
    mndata = MNIST('/home/mumu/Desktop/CSE546/hw1/python-mnist/data/')
    X_train, labels_train = map(np.array, mndata.load_training())
    X_test, labels_test = map(np.array, mndata.load_testing())
    X_train = X_train/255.0
    X_test = X_test/255.0
    return X_test,labels_test,X_train,labels_train

In [3]:
def one_hot_encode(vector):
    n_classes = len(vector.unique())  # 1
    one_hot = torch.zeros((vector.shape[0], n_classes))\
        .type(torch.LongTensor)  # 2
    return one_hot\
        .scatter(1, vector.type(torch.LongTensor).unsqueeze(1), 1)  # 3

In [135]:
X_test,labels_test,X_train,labels_train=load_dataset()
y_train = torch.from_numpy(labels_train)
y_test = torch.from_numpy(labels_test)

y_one_hot = one_hot_encode(y_train)
X_train_tensor = torch.from_numpy(X_train).double()
n_train,d_train = X_train_tensor.shape
X_test_tensor = torch.from_numpy(X_test).double()
n_test,d_test = X_test_tensor.shape

#W = torch.rand(d, 10, requires_grad=True).double()

In [160]:
######################### separate W and b ######################################
h1 = 64 
h2 = 10
# Wx+b => W -> [w_0 W], x -> [1;x] 
W1 = 2/np.sqrt(h1) * torch.rand( h1 , d_train ).T.double() -  1/np.sqrt(h1) # uiformly between (-1/sqrt(m), 1/sqrt(m))
b1 = ones = torch.ones(h1,1).double()
W2 = 2/np.sqrt(h2) * torch.rand( h1 , h2 ).double() -  1/np.sqrt(h2) # uiformly between (-1/sqrt(m), 1/sqrt(m))
b2 = ones = torch.ones(h2,1).double()

print(W1.shape)
print(W2.shape)

torch.Size([784, 64])
torch.Size([64, 10])


In [161]:
######################### separate W and b ######################################
epochs=5000
learning_rate = .01
parms = [W1,b1,W2,b2]#list(list(W2)) #+list(W2))
optimizer = torch.optim.Adam(parms, lr=learning_rate)
W1.requires_grad = True
W2.requires_grad = True
b1.requires_grad = True
b2.requires_grad = True

In [ ]:
######################### separate W and b ######################################
for epoch in range(epochs):
    # fc1: #64-by-#observation 
    fc1 = F.relu (torch.matmul(W1.double().T,X_train_tensor.T) + b1 ) 

    y_hat =(torch.matmul(W2.double().T, fc1)+b2).T
    # cross entropy combines softmax calculation with NLLLoss

    loss = torch.nn.functional.cross_entropy(y_hat, y_train.long())
    
    # before loss.backwards()
    optimizer.zero_grad()
    
    # computes derivatives of the loss with respect to W
    loss.backward()
    
    optimizer.step()
    
    ##### getting training accuracy ############
    a= (torch.argmax(y_hat,axis=1) - y_train)
    train_accuracy =  a[a==0].shape[0] / n_train
    if (epoch % 10 == 0) :
        print ("training accuracy is {}".format(train_accuracy))
    if train_accuracy >0.99:
        print ("final training accuracy is {}".format(train_accuracy))
        break
#     if epoch % 10 == 0 :
#         print('{},\t{:.2f}'.format(epoch, loss.item()))
#         print("the max of the {} the iteration is {}".format(epoch,torch.max(torch.abs(W1.grad))))
#         print("the max of the {} the iteration is {}".format(epoch,torch.max(torch.abs(W2.grad))))
#     if ((torch.max(torch.abs((W1.grad))) < 0.001 ) and torch.max(torch.abs((W2.grad))) < 0.001 ) :
#         break


##########################################################################
################### evaluate it on the test set ##########################
##########################################################################
fc1_test = F.relu (torch.matmul(W1.double().T,X_test_tensor.T) + b1 ) 
y_test_hat =(torch.matmul( W2.double().T,fc1_test)+b2).T
a= (torch.argmax(y_test_hat,axis=1) - y_test)
test_accuracy =  a[a==0].shape[0] / n_test
print(test_accuracy)

training accuracy is 0.09795
training accuracy is 0.7749166666666667
training accuracy is 0.8702666666666666
training accuracy is 0.89285
training accuracy is 0.9081333333333333
training accuracy is 0.91885
training accuracy is 0.9261833333333334
training accuracy is 0.9312333333333334
training accuracy is 0.9356833333333333
training accuracy is 0.9391666666666667
training accuracy is 0.9426333333333333
training accuracy is 0.94565
training accuracy is 0.9483333333333334
training accuracy is 0.9511166666666667
training accuracy is 0.9532333333333334
training accuracy is 0.9554
training accuracy is 0.9572
training accuracy is 0.9586333333333333
training accuracy is 0.9602833333333334
training accuracy is 0.9615166666666667
training accuracy is 0.9628
training accuracy is 0.9642166666666667


In [159]:
##########################################################################
################### evaluate it on the test set ##########################
##########################################################################
fc1_test = F.relu (torch.matmul(W1.double().T,X_test_tensor.T) + b1 ) 
y_test_hat =(torch.matmul( W2.double().T,fc1_test)+b2).T
a= (torch.argmax(y_test_hat,axis=1) - y_test)
test_accuracy =  a[a==0].shape[0] / n_test
print(test_accuracy)

0.9688


torch.Size([60000, 1])

In [110]:
y_train.shape[0]

60000